# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis import core_stops_psth, get_stops_utils, psth_postprocessing, psth_stats

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
# pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
#     pn.planning_data_by_point)
#pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

if not hasattr(pn, 'spikes_df'):
    pn.retrieve_or_make_monkey_data()
    pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted,
                                                            sampling_rate=pn.sampling_rate)

# NEXT: try stop end time instead

# Get captures

In [ ]:
reload(get_stops_utils)

In [ ]:
# Example wiring (mirrors your original usage)
valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    min_stop_duration=0.02,
    max_stop_duration=1.0,
    capture_match_window=0.3,
    distance_thresh=25.0,
    distance_col="distance_from_ff_to_stop",
)


In [ ]:
valid_captures_df

In [ ]:
valid_captures_df

In [ ]:
filtered_no_capture_stops_df

# Get misses

##  one

In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]

pn.make_one_stop_w_ff_df()
one_stop_miss_df = pn.one_stop_w_ff_df[['first_stop_point_index', 'first_stop_time', 'latest_visible_ff', 'ff_distance', 'min_distance_from_adjacent_stops']].copy()
one_stop_miss_df.rename(columns={'first_stop_point_index': 'stop_point_index', 'first_stop_time': 'stop_time'}, inplace=True)
one_stop_miss_df[columns_to_add] = pn.monkey_information.loc[one_stop_miss_df['stop_point_index'], columns_to_add].values

## more

In [ ]:
pn.get_try_a_few_times_info()
pn.get_give_up_after_trying_info()


In [ ]:
unique_stops_df

In [ ]:
columns_to_add = ["stop_id", "stop_id_duration", "stop_id_start_time", "stop_id_end_time"]
shared_columns = ["stop_point_index", "stop_time"] + columns_to_add

# --- Build expanded + ordered tables for GUAT / TAFT ---
GUAT_expanded = get_stops_utils._expand_trials(pn.GUAT_trials_df, pn.monkey_information)
TAFT_expanded = get_stops_utils._expand_trials(pn.TAFT_trials_df, pn.monkey_information)

# add stop_id to GUAT_trials_df and TAFT_trials_df
GUAT_expanded[columns_to_add] = pn.monkey_information.loc[GUAT_expanded['stop_point_index'], columns_to_add].values
TAFT_expanded[columns_to_add] = pn.monkey_information.loc[TAFT_expanded['stop_point_index'], columns_to_add].values


GUAT = get_stops_utils._add_cluster_ordering(GUAT_expanded)
TAFT = get_stops_utils._add_cluster_ordering(TAFT_expanded)

# --- Per-cluster slices (consistent, vectorized) ---
# First stop in each cluster
GUAT_first = GUAT[GUAT["is_first"]].reset_index(drop=True)
TAFT_first = TAFT[TAFT["is_first"]].reset_index(drop=True)

# Last stop in each cluster
GUAT_last = GUAT[GUAT["is_last"]].reset_index(drop=True)
TAFT_last = TAFT[TAFT["is_last"]].reset_index(drop=True)

# Middle stops (exclude first and last)
GUAT_middle = GUAT[GUAT["is_middle"]].reset_index(drop=True)
TAFT_middle = TAFT[TAFT["is_middle"]].reset_index(drop=True)

# “First several” = all but the last stop in each cluster
GUAT_nonfinal = GUAT[GUAT["order_in_cluster"] < GUAT["cluster_size"] - 1].reset_index(drop=True)
TAFT_nonfinal = TAFT[TAFT["order_in_cluster"] < TAFT["cluster_size"] - 1].reset_index(drop=True)

# Combine the “first several” from both, keep only columns you care about, then sort by index
both_nonfinal = (
    pd.concat(
        [
            GUAT_nonfinal[shared_columns],
            TAFT_nonfinal[shared_columns],
        ],
        ignore_index=True
    )
    .sort_values("stop_point_index")
    .reset_index(drop=True)
)

both_first = pd.concat([GUAT_first[shared_columns], 
                         TAFT_first[shared_columns]])

both_middle = pd.concat([GUAT_middle[shared_columns], 
                         TAFT_middle[shared_columns]])

# Optional: if you also want “last several” (all but the first), it’s symmetrical:
# GUAT_last_several = GUAT[GUAT["order_in_cluster"] > 0].reset_index(drop=True)
# TAFT_last_several = TAFT[TAFT["order_in_cluster"] > 0].reset_index(drop=True)

GUAT_last_plus_single_miss = pd.concat([GUAT_last[shared_columns], 
                                         one_stop_miss_df[shared_columns]])

all_misses = pd.concat([one_stop_miss_df[shared_columns], 
                                         GUAT_expanded[shared_columns],
                                         TAFT_nonfinal[shared_columns]
                                         ])

# add stop_end_time

In [ ]:
all_misses

# SELECT

In [ ]:
# capture vs no capture
captures_df=valid_captures_df
no_capture_stops_df=filtered_no_capture_stops_df

captures_df_name = 'valid_captures_df'
no_capture_stops_df_name = 'filtered_no_capture_stops_df'



In [ ]:
# persist vs give up
captures_df=both_nonfinal
no_capture_stops_df=GUAT_last

captures_df_name = 'both_nonfinal'
no_capture_stops_df_name = 'GUAT_last'

In [ ]:
# persist (shrunken) vs give up
captures_df=both_middle
no_capture_stops_df=GUAT_last

captures_df_name = 'both_middle'
no_capture_stops_df_name = 'GUAT_last'

In [ ]:
# persist vs give up (expanded)
captures_df=both_nonfinal
no_capture_stops_df=GUAT_last_plus_single_miss

captures_df_name = 'both_nonfinal'
no_capture_stops_df_name = 'GUAT_last_plus_single_miss'


In [ ]:
# persist (shrunken) vs give up (expanded)
captures_df=both_nonfinal
no_capture_stops_df=GUAT_last_plus_single_miss

captures_df_name = 'both_nonfinal'
no_capture_stops_df_name = 'GUAT_last_plus_single_miss'


In [ ]:
# give up in single miss and give up in GUAT
captures_df=one_stop_miss_df
no_capture_stops_df=GUAT_last

captures_df_name = 'one_stop_miss_df'
no_capture_stops_df_name = 'GUAT_last'


In [ ]:
# give up in single miss and not give up in both_first
captures_df=one_stop_miss_df
no_capture_stops_df=both_first

captures_df_name = 'one_stop_miss_df'
no_capture_stops_df_name = 'both_first'


In [ ]:
# GUAT first several vs TAFT first several
captures_df=GUAT_nonfinal
no_capture_stops_df=TAFT_nonfinal

captures_df_name = 'GUAT_nonfinal'
no_capture_stops_df_name = 'TAFT_nonfinal'


In [ ]:
# GUAT last vs TAFT last
captures_df=GUAT_last
no_capture_stops_df=TAFT_last

captures_df_name = 'GUAT_last'
no_capture_stops_df_name = 'TAFT_last'


In [ ]:
# capture (not in TAFT) vs TAFT last
valid_captures_df_subset = valid_captures_df[~valid_captures_df['stop_id'].isin(TAFT_last['stop_id'])].copy()

captures_df=valid_captures_df_subset
no_capture_stops_df=TAFT_last

captures_df_name = 'valid_captures_df_subset'
no_capture_stops_df_name = 'TAFT_last'

In [ ]:
# all captures vs all misses
captures_df=valid_captures_df
no_capture_stops_df=all_misses

captures_df_name = 'valid_captures_df'
no_capture_stops_df_name = 'all_misses'


In [ ]:
# all misses vs non-capture stops
non_capture_sub = filtered_no_capture_stops_df[~filtered_no_capture_stops_df['stop_id'].isin(all_misses['stop_id'])].copy()

captures_df=non_capture_sub
no_capture_stops_df=all_misses

captures_df_name = 'non_capture_sub'
no_capture_stops_df_name = 'all_misses'



In [ ]:
# all first misses vs non-capture stops
non_capture_sub = filtered_no_capture_stops_df[~filtered_no_capture_stops_df['stop_id'].isin(all_misses['stop_id'])].copy()

all_first_misses = pd.concat([one_stop_miss_df[shared_columns], 
                                         GUAT_first[shared_columns],
                                         TAFT_first[shared_columns]
                                         ])


captures_df=non_capture_sub
no_capture_stops_df=all_first_misses

captures_df_name = 'non_capture_sub'
no_capture_stops_df_name = 'all_first_misses'



# run class

In [ ]:
print(f'{captures_df_name} vs {no_capture_stops_df_name}')
print('=====================================================')


captures_df['stop_time'] = captures_df['stop_id_end_time']
no_capture_stops_df['stop_time'] = no_capture_stops_df['stop_id_end_time']


cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.05,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

an = core_stops_psth.create_psth_around_stops(pn.spikes_df, pn.monkey_information, pn.ff_caught_T_new, cfg,
                                                 captures_df=captures_df,
                                                 no_capture_stops_df=no_capture_stops_df
                                                 )

fig2 = an.plot_comparison(cluster_idx=0)

plt.show()

# Stats in early post-stop window
stats_ = an.statistical_comparison(time_window=(0.0, 0.5))




windows = {
    "pre_bump(-0.3–0.0)": (-0.3, 0.0),
    "early_dip(0.0–0.3)": (0.0, 0.3),
    "late_rebound(0.3–0.8)": (0.3, 0.8),
}
    
summary = psth_postprocessing.compare_windows(an, windows, alpha=0.05)
summary_sub = summary[summary['sig_FDR']].copy()
df = summary_sub.sort_values(["sig_FDR", "window", "p"], ascending=[False, True, True])
df


In [ ]:


windows = {
    "pre_baseline(-0.5–-0.2)": (-0.5, -0.2),
    "pre_bump(-0.2–-0.1)": (-0.2, -0.1),
    "pre_bump2(-0.1–0.0)": (-0.1, 0.0),
    # "pre_bump3(-0.2--0.0)": (-0.2, -0.0),
    "early_dip(0.0–0.1)": (0.0, 0.1),
    "early_rebound(0.1–0.2)": (0.1, 0.2),
    "early_rebound2(0.2–0.3)": (0.2, 0.3),
    "early_rebound3(0.3–0.4)": (0.3, 0.4),
    "early_rebound4(0.4–0.5)": (0.4, 0.5),
    "late_rebound(0.5–0.8)": (0.5, 0.8),
    "post_tail(0.8–1.2)": (0.8, 1.2),
}

summary = psth_postprocessing.compare_windows(an, windows, alpha=0.05)
summary_sub = summary[summary['sig_FDR']].copy()
df = summary_sub.sort_values(["sig_FDR", "window", "p"], ascending=[False, True, True])
df

In [ ]:
summary.sort_values(["sig_FDR", "p"], ascending=[False, True]).head(20)

# More plots

## Quickly plot PSTHs for the top significant neurons

In [ ]:
import numpy as np

def plot_top_psths(analyzer, summary: pd.DataFrame, epoch: str, top_k=6):
    # pick significant clusters in the epoch, ranked by |d|
    g = summary[(summary["window"] == epoch) & (summary["sig_FDR"])].copy()
    if g.empty:
        print(f"No significant clusters for {epoch}."); return
    g = g.sort_values("cohens_d", key=lambda s: s.abs(), ascending=False).head(top_k)

    # map string cluster ids back to analyzer cluster indices
    plotted = 0
    for cl_str in g["cluster"]:
        # analyzer.clusters holds original IDs (numeric or str)
        # coerce both sides to string for robust matching
        matches = np.where(np.array(list(map(str, analyzer.clusters))) == str(cl_str))[0]
        if len(matches) == 0: 
            continue
        ci = int(matches[0])
        analyzer.plot_comparison(cluster_idx=ci)  # your existing method
        plotted += 1
    if plotted == 0:
        print("Nothing plotted (no matches).")

# usage
plot_top_psths(an, summary, "early_dip(0.0–0.3)", top_k=7)


## Heatmap of effect sizes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_sig_heatmap(summary: pd.DataFrame, title="Significant effects (Cohen's d)"):
    # keep only FDR-significant rows
    sig = summary[summary["sig_FDR"]].copy()
    if sig.empty:
        print("No significant results to plot.")
        return

    # pivot to clusters × windows, values = d
    pivot = sig.pivot_table(index="cluster", columns="window", values="cohens_d", aggfunc="mean")

    # optional: sort clusters by strongest absolute effect
    order = np.argsort(-pivot.abs().max(axis=1).values)
    pivot = pivot.iloc[order]

    # plot
    fig, ax = plt.subplots(figsize=(8, max(3, 0.35 * len(pivot))))
    im = ax.imshow(pivot.values, aspect="auto", cmap="coolwarm", vmin=-np.nanmax(abs(pivot.values)), vmax=np.nanmax(abs(pivot.values)))
    ax.set_xticks(range(pivot.shape[1])); ax.set_xticklabels(pivot.columns, rotation=30, ha="right")
    ax.set_yticks(range(pivot.shape[0])); ax.set_yticklabels(pivot.index)
    ax.set_title(title)
    cbar = plt.colorbar(im, ax=ax); cbar.set_label("Cohen's d (capture − miss)")
    plt.tight_layout()
    plt.show()

# usage
plot_sig_heatmap(summary)


## Bar chart

In [ ]:
# keep only rows where sig_FDR is True
sig_rows = summary[summary["sig_FDR"]]

# plot effect sizes by epoch
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15,5))
sns.barplot(data=sig_rows, x="window", y="cohens_d", hue="cluster", dodge=True)
plt.axhline(0, color="k", lw=1)
plt.ylabel("Cohen's d (capture − miss)")
# change angle of xticks
plt.xticks(rotation=30)
plt.title("Significant neurons across epochs")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


## Bar chart of significant effects per epoch (one bar per cluster)

In [ ]:
def plot_sig_bars(summary: pd.DataFrame, epoch: str):
    g = summary[(summary["window"] == epoch) & (summary["sig_FDR"])].copy()
    if g.empty:
        print(f"No significant clusters for {epoch}."); return
    g = g.sort_values("cohens_d", key=lambda s: s.abs(), ascending=False)

    fig, ax = plt.subplots(figsize=(10, max(3, 0.35 * len(g))))
    ax.barh(g["cluster"], g["cohens_d"])
    ax.axvline(0, color="k", lw=1, alpha=0.5)
    ax.set_xlabel("Cohen's d (capture − miss)")
    ax.set_ylabel("Cluster")
    ax.set_title(f"Significant clusters in {epoch}")
    plt.tight_layout(); plt.show()

# usage
plot_sig_bars(summary, "pre_bump(-0.3–0.0)")
plot_sig_bars(summary, "early_dip(0.0–0.3)")
plot_sig_bars(summary, "late_rebound(0.3–0.8)")


# Appendix

## Other anchors for stops
https://docs.google.com/document/d/1nJOQlqsyq7eOVtUlL8TAAacC25vXeQvlGfDI-SbQrKE/edit?tab=t.0

In [ ]:
import numpy as np
import pandas as pd

def _finite_diff(y, t):
    dy = np.gradient(y, t)  # robust to uneven dt
    return dy

def anchor_peak_decel(df, pre_window_s=0.5):
    # peak negative acceleration in a window ending at stop onset
    # requires per-stop onset; derive from first time of each stop_id
    onset = df.groupby("stop_id")["time"].min()
    t = df["time"].to_numpy()
    a = _finite_diff(df["monkey_speed"].to_numpy(), t)  # acceleration
    df2 = df.assign(accel=a)
    rows = []
    for sid, t0 in onset.items():
        m = (df2["stop_id"]==sid) & (df2["time"].between(t0-pre_window_s, t0, inclusive="left"))
        if not m.any(): 
            continue
        i = df2.loc[m, "accel"].idxmin()
        rows.append((sid, df2.at[i, "time"]))
    out = pd.DataFrame(rows, columns=["stop_id","anchor_time"])
    out["anchor_kind"] = "peak_decel"
    return out





## other processing/windows

In [ ]:

# # Per-cluster plots with bands
# fig1 = an.plot_psth(cluster_idx=None, show_individual=False)


# df = psth_postprocessing.export_psth_to_df(an)              # all clusters
# df_c0 = psth_postprocessing.export_psth_to_df(an, [0])      # just the first cluster


# windows = {
#     "pre_bump(-0.3–0.0)": (-0.3, 0.0),
#     "early_dip(0.0–0.3)": (0.0, 0.3),
#     "late_rebound(0.3–0.8)": (0.3, 0.8),
# }


# windows = {
#     "pre_baseline(-0.6–-0.3)": (-0.6, -0.3),
#     "pre_bump(-0.3–0.0)": (-0.3, 0.0),
#     "early_dip(0.0–0.1)": (0.0, 0.1),
#     "early_rebound(0.1–0.3)": (0.1, 0.3),
#     "late_rebound(0.3–0.8)": (0.3, 0.8),
#     "post_tail(0.8–1.5)": (0.8, 1.5),
# }

## Validate near-miss single stops

In [ ]:
# Check whether "near-miss" stops (one_stop_w_ff_df) are truly not part of a stop cluster.  
# A stop cluster is defined as ≥ 2 stops where each consecutive stop is within 50 cm (cumulative distance).

# --- Step 1: Create one-stop dataframe and assign cluster IDs
pn.make_one_stop_w_ff_df()
pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information)

# --- Step 2: Build stop-cluster summary
stop_cluster_df = (
    pn.monkey_information.loc[pn.monkey_information['whether_new_distinct_stop'], ['point_index', 'stop_cluster_id']]
    .copy()
)
stop_cluster_df['num_stops_in_cluster'] = (
    stop_cluster_df.groupby('stop_cluster_id')['point_index'].transform('count')
)

# --- Step 3: Merge cluster info into one-stop dataframe (if not already present)
if 'stop_cluster_id' not in pn.one_stop_w_ff_df.columns:
    pn.one_stop_w_ff_df = pn.one_stop_w_ff_df.merge(
        stop_cluster_df.rename(columns={'point_index': 'first_stop_point_index'}),
        on='first_stop_point_index',
        how='left'
    )

# --- Step 4: Inspect any one-stop rows that actually fall in a multi-stop cluster
pn.one_stop_w_ff_df[pn.one_stop_w_ff_df['num_stops_in_cluster'] > 1]


## Check dt between stops in clusters

In [ ]:
import pandas as pd


# Expand so each stop_index gets its own row
GUAT_expanded = pn.GUAT_trials_df.explode("stop_indices").reset_index(drop=True)

# Optionally rename column
GUAT_expanded = GUAT_expanded.rename(columns={"stop_indices": "stop_point_index"})
GUAT_expanded['stop_time'] = pn.monkey_information['time'].loc[GUAT_expanded['stop_point_index']].values

TAFT_expanded = pn.TAFT_trials_df.explode("stop_indices").reset_index(drop=True)

# Optionally rename column
TAFT_expanded = TAFT_expanded.rename(columns={"stop_indices": "stop_point_index"})
TAFT_expanded['stop_time'] = pn.monkey_information['time'].loc[TAFT_expanded['stop_point_index']].values


# group TAFT_expanded by stop_cluster_id and drop the last row of each group
TAFT_expanded.sort_values('stop_point_index', inplace=True)
TAFT_nonfinal = TAFT_expanded.groupby('stop_cluster_id').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)


In [ ]:
GUAT_expanded['dt'] = GUAT_expanded['stop_time'].diff()
TAFT_expanded['dt'] = TAFT_expanded['stop_time'].diff()
GUAT_sub = GUAT_expanded[GUAT_expanded['dt'] < 0.5]
TAFT_sub = TAFT_expanded[TAFT_expanded['dt'] < 0.5]
GUAT_sub


In [ ]:
sns.histplot(GUAT_sub[['dt']])
plt.show()

In [ ]:
sns.histplot(TAFT_sub[['dt']])
plt.show()

In [ ]:
GUAT_sub[['dt']].describe()

In [ ]:
TAFT_sub[['dt']].describe()

## check inter-stop intervals

In [ ]:
reload(process_monkey_information)

In [ ]:
pn.monkey_information = process_monkey_information.add_whether_new_distinct_stop_and_stop_id(pn.monkey_information)

In [ ]:
pn.monkey_information

In [ ]:

unique_stops_df = (
    pn.monkey_information[['stop_id', 'point_index', 'time', 'stop_id_start_time', 'stop_id_end_time', 'stop_id_duration']].groupby("stop_id", as_index=False, sort=False)
    .first()
    .reset_index(drop=True)
)
unique_stops_df

In [ ]:
unique_stops_df = get_stops_utils.extract_unique_stops(pn.monkey_information)
unique_stops_df

In [ ]:
unique_stops_df = get_stops_utils.extract_unique_stops(pn.monkey_information)
onsets = unique_stops_df['time'].to_numpy()

In [ ]:
returned = get_stops_utils.plot_inter_stop_intervals(onsets)
ax1 = returned['ax1']

# add an additional vertical line to the linear plot
additional_vline = 0.3
ax1.axvline(additional_vline, linestyle="--", color="b", alpha=0.8, label=f"x = {additional_vline}s")
ax1.legend()
plt.show()

## check stop durations

In [ ]:
plt.figure()
ax = sns.histplot(unique_stops_df['stop_id_duration'], bins=100)
ax.set_xlim(0, 1)
plt.show()

## add stop_id

In [ ]:
pn.monkey_information = process_monkey_information.add_whether_new_distinct_stop_and_stop_id(pn.monkey_information)
pn.monkey_information

## stop clusters

In [ ]:
pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information, max_cluster_distance=50, use_ff_caught_time_new_to_separate_clusters=False,
                                                                        col_exists_ok=False)

In [ ]:
for max_cluster_distance in [50, 45, 40, 35, 30, 20, 50]:
    print(f'max_cluster_distance: {max_cluster_distance}')
    pn.monkey_information = find_GUAT_or_TAFT_trials.add_stop_cluster_id(pn.monkey_information, max_cluster_distance=max_cluster_distance, use_ff_caught_time_new_to_separate_clusters=False,
                                                                        col_exists_ok=False)

    stop_sub = pn.monkey_information[pn.monkey_information['stop_id'].notna()].copy()
    stop_sub = stop_sub[['stop_id', 'stop_cluster_id']].drop_duplicates()
    # Compute cluster sizes properly
    stop_sub['cluster_size'] = stop_sub.groupby('stop_cluster_id')['stop_id'].transform('size')

    # Show how many rows belong to clusters of size > 1
    print('number of clusters: ', stop_sub['stop_cluster_id'].unique().shape[0])
    print('number of stops: ', stop_sub['stop_id'].unique().shape[0])
    print('number of clusters with more than 1 stop: ', stop_sub[stop_sub['cluster_size'] > 1].shape[0])
    print('--------------------------------')


In [ ]:
num_stops = pn.monkey_information['stop_id'].unique().shape[0]
num_clusters = pn.monkey_information['stop_cluster_id'].unique().shape[0]
num_clusters

## censor_mask_for_event

censor_mdef censor_mask_for_event(t0, all_stops, time_axis, pad=0.15):
    abs_times = t0 + time_axis
    dmin = np.min(np.abs(abs_times[:,None] - all_stops[None,:]), axis=1)
    keep = (dmin >= pad) | np.isclose(abs_times, t0)
    return keep  # bool, shape (len(time_axis),)
ask_for_event